## Scraper for r/jokes subreddit that will be used to train NLP model

In [5]:
import pandas as pd
import praw

In [12]:
import r_auth

reddit = praw.Reddit(client_id = r_auth.client_id,
                     client_secret = r_auth.client_secret,
                     user_agent = r_auth.user_agent,
                     username = r_auth.username,
                     password = r_auth.password)


subreddit = reddit.subreddit('Jokes')